In [1]:
import tensorflow as tf 
import pandas as pd 


In [4]:
df = pd.read_csv('Train.csv', encoding = 'cp949')

df.head()

,Movie,Playdate,Play_month,Weekdays,Rep_Nationality,Grade,Rep_production,Rep_distributor,Rep_genre,Rep_director,...,Week2_sales_share,Week2_audience,Week2_screen,Week2_screening,Week3_holiday,Week4_holiday,Week2_Cum_audience,Week3_Cum_audience,Week4_Cum_audience,Week5_Cum_audience
0,명량,2014-07-30,7,수요일,한국,15세이상관람가,㈜빅스톤픽처스,씨제이이앤엠(주),액션,김한민,...,0.484600,4814896,2185,46772,3,2,12122075,15540940,16556133,17034795
1,도둑들,2012-07-25,7,수요일,한국,15세이상관람가,(주)케이퍼필름,(주)쇼박스,범죄,최동훈,...,0.436286,3488656,1737,34679,3,2,7837525,10079961,11481440,12270974
2,베테랑,2015-08-05,8,수요일,한국,15세이상관람가,(주)외유내강,씨제이이앤엠(주),액션,류승완,...,0.475000,3491532,1115,36383,2,2,7458012,9579065,11176810,12048616
3,암살,2015-07-22,7,수요일,한국,15세이상관람가,(주)케이퍼필름,(주)쇼박스,액션,최동훈,...,0.326000,2960563,1094,32184,2,2,7647796,9491708,11015334,11793736
4,국제시장,2014-12-17,12,수요일,한국,12세이상관람가,㈜제이케이필름 JK Film,씨제이이앤엠(주),드라마,윤제균,...,0.392800,3032951,1821,28139,3,2,5312629,8329563,10146564,11405561


## 변수 설명
1. Movie : 영화 제목

2. Playdate : 개봉 날짜
   Play_month : 개봉 월
   Weekdays : 개봉 요일

3. Rep_Nationality : 대표 국가

4. Grade : 관람 등급

5. Rep_production : 대표 제작사
   Prod_grade : 제작사 등급
   Prod_avg_grade : 제작사 평균 등급
                    (제작사는 시나리오 선정, 투자자 물색, 배우 캐스팅 등의 역할을 맡는다.
                     따라서 배우 등급, 배급사 등급, 감독 등급, 영화 평점 등을 반영하여 계산하였다.)

6. Rep_distribution : 대표 배급사
   Dist_grade : 배급사 등급
                (우리나라의 경우 배급사가 메인 투자자인 경우가 많으며, 배급사가 스크린을 얼마나 확보하느냐에 따라 영화의 성과가 결정된다.
                 따라서 배급사에 따른 개봉 첫째주의 스크린 수를 파악하여 이를 배급사의 등급으로 대체하였다.)

7. Rep_genre : 대표 장르

8. Rep_director : 대표 감독
   Direc_avg_grade : 감독 평균 등급
                     (감독 등급은 2년간 감독한 영화의 평균 평점으로 대체하였다.

9. Rep_actor1 : 대표 배우1
   Actor1_grade : 대표 배우1 등급
   Rep_actor2 : 대표 배우2
   Actor2_grade : 대표 배우2 등급
   Rep_actor3 : 대표 배우3
   Actor3_grade : 대표 배우3 등급
   Rep_actor4 : 대표 배우4
   Actor4_grade : 대표 배우4 등급
   Rep_actor5 : 대표 배우5
   Actor5_grade : 대표 배우5 등급
                  (배우의 등급은 2년간 배우가 출연한 영화들의 2주차 평균 누적관객수를 변환한 값이다.)

10. Critic_star : 평론가 평점

11. Netizen_star_before_play : 개봉 전 네티즌 평점
    Num_of_participants_before_play : 개봉 전 평가 참여 인원

12. Netizen_star_after_play : 개봉 후 네티즌 평점

13. Diff_netizen_star : 개봉 전/후 평점 차이

14. Expect_index_wannasee : 개봉 전 기대지수(보고싶어요)
    Expect_index_notreally : 개봉 전 기대지수(글쎄요)
    Ratio_notreally : 글쎼요 비율

15. Week1_isplay : 1주차 상영 여부
    Week1_playdays : 1주차 상영일수
    Week1_holiday : 1주차 공휴일수(주말 포함)
    Week1_sales : 1주차 매출액
    Week1_sales_share : 1주차 매출액 점유율
    Week1_audience : 1주차 관객수
    Week1_screen : 1주차 스크린수
    Week1_Screening : 1주차 상영횟수

## 과제 

킹스맨2  0927 개봉
남한산성 0930 개봉
넛잡 1003 개봉


**데이터를 기준으로 킹스맨2, 남한산성 => 1주차 데이터 안 스고 2주차 누적 관객 맞추기, 넛잡 => 1주차 데이터 안쓰고 1주차 누적관객수 맞추기 **

각각에 대한 모델링을 따로 할필요가 있다. 



In [6]:
#파생변수 x, 9개의 변수만을 이용
#1주차 관객수 예측 모델 
col_list = ['Play_month', 'Critic_star', 'Netizen_star_before_play', 'Num_of_participants_before_play','Expect_index_wannasee', 'Expect_index_notreally',
           'Week1_isplay','Week1_playdays','Week1_holiday','Week1_audience']
df_1 = df[col_list]


In [8]:
df_1.head()

,Play_month,Critic_star,Netizen_star_before_play,Num_of_participants_before_play,Expect_index_wannasee,Expect_index_notreally,Week1_isplay,Week1_playdays,Week1_holiday,Week1_audience
0,7,6.0,9.0,1676.0,13639.0,1126.0,1,8,2,7307179
1,7,7.0,9.0,703.0,NaN,NaN,1,8,2,4348869
2,8,8.0,9.0,688.0,9952.0,1339.0,1,8,2,3966480
3,7,7.0,8.0,917.0,20846.0,3541.0,1,8,2,4687233
4,12,6.0,9.0,1233.0,12784.0,2052.0,1,8,2,2279678


In [9]:
# parameter 설정 
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

#netowrk parameter 설정 
n_hidden_1 = 128
n_hidden_2 = 128
n_hidden_3 = 128

n_input = 9

In [ ]:
#graph 구성 
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, 1])

